# Traditional Models

For: Tan Cheen Hao!!!

The idea for this file is to train multiple traditional models given the dataset. The data files you will need to import is unfortunately not ready. But for now, write and test the code using `model_building_data.csv` which is provided in the data folder. Keep in mind that the final training/testing files will have more fields.

You should test out everything you can think of. Focus on regression models but some classification models could be useful during stacking (up or down prediction rather than the exact value).

Also, these models will not be timeseries models. Therefore, you will need to engineer lagged parameters to make use of the timeseries nature.

Some models will be use during the stacking process. If not then it will be use as benchmark models. Everything will be included in the report so the more the better. Keep in mind that different models will require a different processing pipeline. This is because some models can take care of missing data, some cant. Some models are more sensitive to data ranges. Use GPT to check how to process for each! A useful tip is if you can generalise a preprocessing function for every model, then your life will be a lot easier.

Last thing to keep in mind is, some rows might have missing revenue but non-missing CAR etc. If you will drop NaNs, drop for each y values differently to prevent unnecessary data loss.

Tune all parameters for all models using 3-fold CV with the timesplit function like in assignment 1. I'll write a different time split function and we'll run with 5-10 fold CV again later before submission.

This file should save the output of the prediction in the format:

| ticker | quarter_year  | log_revenue_prediction_svm_regres | CAR_prediction_svm_regres | log_revenue_prediction_linear_regres | CAR_prediction_linear_regres |
|--------|---------------|-----------------------------------|---------------------------|--------------------------------------|------------------------------|
| BAC    | Q1 2001       | 123                               | 0.5                       | 123                                  | 0.5                          |
| JPM    | Q1 2001       | 456                               | 0.8                       | 456                                  | 0.8                          |
| WFC    | Q1 2001       | 789                               | 0.25                      | 789                                  | 0.25                         |

Enjoy!